# LLM Reflection

AutoGen supports different LLM prompting and reasoning strategies, such as ReAct, Reflection/Self-Critique, and more. This notebook demonstrates how to realize general LLM reflection with AutoGen. Reflection is a general prompting strategy which involves having LLMs analyze their own outputs, behaviors, knowledge, or reasoning processes.

This example leverages a generic interface [nested chats](/docs/tutorial/conversation-patterns#nested-chats) to implement the reflection using multi-agents.

First make sure the `pyautogen` package is installed.

In [6]:
import os

llm_config={
    "config_list" : [{
        "api_type" : "groq",
        "model" : "llama3-70b-8192",  # "mixtral-8x7b-32768",
        "api_key" : os.environ.get("GROQ_API_KEY"), 
        }]
    }


In [7]:
#! pip install "pyautogen>=0.2.18"

Import the relevant modules and configure the LLM.
See [LLM Configuration](/docs/topics/llm_configuration) for how to configure LLMs.

In [8]:
import os

from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor

config_list = [
    #{"model": "gpt-4-1106-preview", "api_key": os.environ["OPENAI_API_KEY"]},
    #{"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]},
    {"api_type" : "groq", "model" : "llama3-70b-8192", "api_key" : os.environ.get("GROQ_API_KEY"),}
    
]
# You can also use the following method to load the config list from a file or environment variable.
# config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

## Construct Agents
Now we create three agents, including `user_proxy` as a user proxy, `writing_assistant` for generating solutions (based on the initial request or critique), and `reflection_assistant` for reflecting and providing critique.

In [9]:
os.makedirs("coding", exist_ok=True)
# Use DockerCommandLineCodeExecutor if docker is available to run the generated code.
# Using docker is safer than running the generated code directly.
# code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
code_executor = LocalCommandLineCodeExecutor(work_dir="coding")

user_proxy = UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    code_execution_config={"executor": code_executor},
)

writing_assistant = AssistantAgent(
    name="writing_assistant",
    system_message="You are an writing assistant tasked to write engaging blogpost. You try generate the best blogpost possible for the user's request. If the user provides critique, respond with a revised version of your previous attempts.",
    llm_config={"config_list": config_list, "cache_seed": None},
)

reflection_assistant = AssistantAgent(
    name="reflection_assistant",
    system_message="Generate critique and recommendations on the writing. Provide detailed recommendations, including requests for length, depth, style, etc..",
    llm_config={"config_list": config_list, "cache_seed": None},
)

## Construct Agent Chats with `reflection_assistant` being a Nested Agent for Reflection

In [10]:
def reflection_message(recipient, messages, sender, config):
    print("Reflecting...")
    return f"Reflect and provide critique on the following writing. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


nested_chat_queue = [
    {
        "recipient": reflection_assistant,
        "message": reflection_message,
        "max_turns": 1,
    },
]
user_proxy.register_nested_chats(
    nested_chat_queue,
    trigger=writing_assistant,
    # position=4,
)

# Use Cache.disk to cache the generated responses.
# This is useful when the same request to the LLM is made multiple times.
with Cache.disk(cache_seed=42) as cache:
    user_proxy.initiate_chat(
        writing_assistant,
        message="Write an engaging blogpost on the recent updates in AI. "
        "The blogpost should be engaging and understandable for general audience. "
        "Should have more than 3 paragraphes but no longer than 1000 words.",
        max_turns=2,
        cache=cache,
    )

user_proxy (to writing_assistant):

Write an engaging blogpost on the recent updates in AI. The blogpost should be engaging and understandable for general audience. Should have more than 3 paragraphes but no longer than 1000 words.

--------------------------------------------------------------------------------


writing_assistant (to user_proxy):

**Revolutionizing the Future: Recent Breakthroughs in Artificial Intelligence**

Artificial Intelligence (AI) has come a long way since its inception, and recent updates have taken the world by storm. From improving virtual assistants to creating life-changing healthcare solutions, AI has been rapidly advancing and transforming the way we live and work. In this blog post, we'll delve into the latest developments in AI and explore how they're revolutionizing various industries and aspects of our lives.

**The Rise of Conversational AI**

One of the most significant updates in AI is the rapid improvement of conversational AI. Virtual assistants like Siri, Alexa, and Google Assistant have become increasingly sophisticated, allowing for more natural and human-like interactions. With the ability to understand nuances in language and context, these assistants can now perform tasks more efficiently and accurately. For instance, Google's Duplex AI can even m